# LightGBM Native API on Adult (Census) Data

In [92]:
import os
os.cpu_count()

12

In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay

import time

In [94]:
import sys

sys.path.append('C:/Users/scott/ML/')
from library import data_processing

## 1.  Data Processing

In [95]:
df = pd.read_csv('C:/Users/scott/ML/adult/adult.data', sep = ',\s', header = None)


C:\Users\scott\AppData\Local\Temp\ipykernel_6948\499192434.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('C:/Users/scott/ML/adult/adult.data', sep = ',\s', header = None)


In [96]:
df.shape

(32561, 15)

In [97]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [98]:
test = pd.read_csv('C:/Users/scott/ML/adult/adult_test.csv', sep = ',\s')
test.head()

C:\Users\scott\AppData\Local\Temp\ipykernel_6948\687727845.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  test = pd.read_csv('C:/Users/scott/ML/adult/adult_test.csv', sep = ',\s')


,,,,,,,,,,,,,,|1x3 Cross validator
25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


In [99]:
test.reset_index(inplace = True)

In [100]:
test.shape

(16281, 15)

In [101]:
colnames = ['age', 'workclass', 'fnlwgt', 'educatn', 'edctn_num', 'mrtl_sts', 'occptn', 'rlnshp', 'race', 'sex', 'cgain', 'closs', 'hrsprweek', 'ntv_cntry', 'income']

In [102]:
df.columns = colnames
df.head()

,age,workclass,fnlwgt,educatn,edctn_num,mrtl_sts,occptn,rlnshp,race,sex,cgain,closs,hrsprweek,ntv_cntry,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [103]:
test.columns = colnames
test.head()

,age,workclass,fnlwgt,educatn,edctn_num,mrtl_sts,occptn,rlnshp,race,sex,cgain,closs,hrsprweek,ntv_cntry,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


In [104]:
df.dtypes

age           int64
workclass    object
fnlwgt        int64
educatn      object
edctn_num     int64
mrtl_sts     object
occptn       object
rlnshp       object
race         object
sex          object
cgain         int64
closs         int64
hrsprweek     int64
ntv_cntry    object
income       object
dtype: object

In [105]:
df.describe()

,age,fnlwgt,edctn_num,cgain,closs,hrsprweek
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [106]:
catlist = ['workclass', 'educatn', 'mrtl_sts', 'occptn', 'rlnshp', 'race', 'sex', 'ntv_cntry', 'income']
for v in catlist:
    print(df[v].value_counts())

Private             22696
Self-emp-not-inc     2541
Local-gov            2093
?                    1836
State-gov            1298
Self-emp-inc         1116
Federal-gov           960
Without-pay            14
Never-worked            7
Name: workclass, dtype: int64
HS-grad         10501
Some-college     7291
Bachelors        5355
Masters          1723
Assoc-voc        1382
11th             1175
Assoc-acdm       1067
10th              933
7th-8th           646
Prof-school       576
9th               514
12th              433
Doctorate         413
5th-6th           333
1st-4th           168
Preschool          51
Name: educatn, dtype: int64
Married-civ-spouse       14976
Never-married            10683
Divorced                  4443
Separated                 1025
Widowed                    993
Married-spouse-absent      418
Married-AF-spouse           23
Name: mrtl_sts, dtype: int64
Prof-specialty       4140
Craft-repair         4099
Exec-managerial      4066
Adm-clerical         3770
Sales 

In [107]:
for v in catlist:
    print(df[v].unique().tolist())

['State-gov', 'Self-emp-not-inc', 'Private', 'Federal-gov', 'Local-gov', '?', 'Self-emp-inc', 'Without-pay', 'Never-worked']
['Bachelors', 'HS-grad', '11th', 'Masters', '9th', 'Some-college', 'Assoc-acdm', 'Assoc-voc', '7th-8th', 'Doctorate', 'Prof-school', '5th-6th', '10th', '1st-4th', 'Preschool', '12th']
['Never-married', 'Married-civ-spouse', 'Divorced', 'Married-spouse-absent', 'Separated', 'Married-AF-spouse', 'Widowed']
['Adm-clerical', 'Exec-managerial', 'Handlers-cleaners', 'Prof-specialty', 'Other-service', 'Sales', 'Craft-repair', 'Transport-moving', 'Farming-fishing', 'Machine-op-inspct', 'Tech-support', '?', 'Protective-serv', 'Armed-Forces', 'Priv-house-serv']
['Not-in-family', 'Husband', 'Wife', 'Own-child', 'Unmarried', 'Other-relative']
['White', 'Black', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other']
['Male', 'Female']
['United-States', 'Cuba', 'Jamaica', 'India', '?', 'Mexico', 'South', 'Puerto-Rico', 'Honduras', 'England', 'Canada', 'Germany', 'Iran', 'Philipp

In [108]:
df['workclass'].value_counts()

Private             22696
Self-emp-not-inc     2541
Local-gov            2093
?                    1836
State-gov            1298
Self-emp-inc         1116
Federal-gov           960
Without-pay            14
Never-worked            7
Name: workclass, dtype: int64

In [109]:
for v in catlist:
    print(test[v].unique().tolist())

['Private', 'Local-gov', '?', 'Self-emp-not-inc', 'Federal-gov', 'State-gov', 'Self-emp-inc', 'Without-pay', 'Never-worked']
['11th', 'HS-grad', 'Assoc-acdm', 'Some-college', '10th', 'Prof-school', '7th-8th', 'Bachelors', 'Masters', 'Doctorate', '5th-6th', 'Assoc-voc', '9th', '12th', '1st-4th', 'Preschool']
['Never-married', 'Married-civ-spouse', 'Widowed', 'Divorced', 'Separated', 'Married-spouse-absent', 'Married-AF-spouse']
['Machine-op-inspct', 'Farming-fishing', 'Protective-serv', '?', 'Other-service', 'Prof-specialty', 'Craft-repair', 'Adm-clerical', 'Exec-managerial', 'Tech-support', 'Sales', 'Priv-house-serv', 'Transport-moving', 'Handlers-cleaners', 'Armed-Forces']
['Own-child', 'Husband', 'Not-in-family', 'Unmarried', 'Wife', 'Other-relative']
['Black', 'White', 'Asian-Pac-Islander', 'Other', 'Amer-Indian-Eskimo']
['Male', 'Female']
['United-States', '?', 'Peru', 'Guatemala', 'Mexico', 'Dominican-Republic', 'Ireland', 'Germany', 'Philippines', 'Thailand', 'Haiti', 'El-Salvado

In [110]:
pd.crosstab(df['educatn'],df['edctn_num'])

edctn_num,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
educatn,,,,,,,,,,,,,,,,
10th,0,0,0,0,0,933,0,0,0,0,0,0,0,0,0,0
11th,0,0,0,0,0,0,1175,0,0,0,0,0,0,0,0,0
12th,0,0,0,0,0,0,0,433,0,0,0,0,0,0,0,0
1st-4th,0,168,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5th-6th,0,0,333,0,0,0,0,0,0,0,0,0,0,0,0,0
7th-8th,0,0,0,646,0,0,0,0,0,0,0,0,0,0,0,0
9th,0,0,0,0,514,0,0,0,0,0,0,0,0,0,0,0
Assoc-acdm,0,0,0,0,0,0,0,0,0,0,0,1067,0,0,0,0
Assoc-voc,0,0,0,0,0,0,0,0,0,0,1382,0,0,0,0,0


In [111]:
df['cntry'] = np.where(df['ntv_cntry']== ' United-States', 'USA', 'Non_US')
df['y'] = np.where(df['income']== '>50K', 1, 0)

In [112]:
df['y'].value_counts()

0    24720
1     7841
Name: y, dtype: int64

In [113]:
test['income'].value_counts()

<=50K.    12435
>50K.      3846
Name: income, dtype: int64

In [114]:
test['cntry'] = np.where(test['ntv_cntry']== ' United-States', 'USA', 'Non_US')
test['y'] = np.where(test['income']== '>50K.', 1, 0)

In [115]:
test['y'].value_counts()

0    12435
1     3846
Name: y, dtype: int64

In [116]:
df = df.drop(['educatn', 'ntv_cntry', 'income'], axis = 1)

In [117]:
df.head()

,age,workclass,fnlwgt,edctn_num,mrtl_sts,occptn,rlnshp,race,sex,cgain,closs,hrsprweek,cntry,y
0,39,State-gov,77516,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,Non_US,0
1,50,Self-emp-not-inc,83311,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,Non_US,0
2,38,Private,215646,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,Non_US,0
3,53,Private,234721,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,Non_US,0
4,28,Private,338409,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Non_US,0


In [118]:
test = test.drop(['educatn', 'ntv_cntry', 'income'], axis = 1)

In [119]:
df.replace('?', np.nan, inplace = True)
test.replace('?', np.nan, inplace = True)

In [120]:
df.isna().sum() * 100 / len(df)

age          0.000000
workclass    5.638647
fnlwgt       0.000000
edctn_num    0.000000
mrtl_sts     0.000000
occptn       5.660146
rlnshp       0.000000
race         0.000000
sex          0.000000
cgain        0.000000
closs        0.000000
hrsprweek    0.000000
cntry        0.000000
y            0.000000
dtype: float64

In [121]:
test.isna().sum() * 100 / len(test)

age          0.000000
workclass    5.914870
fnlwgt       0.000000
edctn_num    0.000000
mrtl_sts     0.000000
occptn       5.933296
rlnshp       0.000000
race         0.000000
sex          0.000000
cgain        0.000000
closs        0.000000
hrsprweek    0.000000
cntry        0.000000
y            0.000000
dtype: float64

In [122]:
# LightGBM can natively handle categorical features. For this we need to cast the cattegorical variables as datatype 'category'

df['sex'] = df['sex'].astype('category')
df['workclass'] = df['workclass'].astype('category')
df['mrtl_sts'] = df['mrtl_sts'].astype('category')
df['occptn'] = df['occptn'].astype('category')
df['rlnshp'] = df['rlnshp'].astype('category')
df['race'] = df['race'].astype('category')
df['cntry'] = df['cntry'].astype('category')

In [123]:
cat_features = ['sex', 'workclass', 'mrtl_sts', 'occptn', 'rlnshp', 'race', 'cntry']

In [124]:
df_train, df_val = data_processing.stratified_sampling_with_weight(data = df, weight = 'fnlwgt', test_frac = 0.3, seed = 2025, stratified_col = ['y'])

In [125]:
df_train['fnlwgt'].sum()/df['fnlwgt'].sum()

0.7000109649629018

In [126]:
df_train_kfold = data_processing.kfold_stratified_with_weights(data = df_train, fold_col = 'cv_fold', weight = 'fnlwgt', seed = 2025, stratified_col = ['y'], nFolds = 3)

Fold_1
Fold_2
Fold_3


C:\Users/scott/ML\library\data_processing.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_fold = data_fold.append(df_fold)
C:\Users/scott/ML\library\data_processing.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_fold = data_fold.append(df_fold)
C:\Users/scott/ML\library\data_processing.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_fold = data_fold.append(df_to_split)


## 2. Train LightGBM using native api

In [127]:
columns = df_train_kfold.columns.tolist()
columns

['age',
 'workclass',
 'fnlwgt',
 'edctn_num',
 'mrtl_sts',
 'occptn',
 'rlnshp',
 'race',
 'sex',
 'cgain',
 'closs',
 'hrsprweek',
 'cntry',
 'y',
 'cv_fold']

In [128]:
target = 'y'
cv_col = 'cv_fold'
weight = 'fnlwgt'

othervars = [target, cv_col, weight]
predictors = [v for v in columns if v not in othervars]

In [129]:
df_train = df_train_kfold.reset_index(drop = True)
df_val = df_val.reset_index(drop = True)

In [130]:
#Prepare the data

import lightgbm as lgb

train_data = lgb.Dataset(df_train[predictors], label=df_train[target], weight = df_train[weight], categorical_feature= cat_features, free_raw_data=False)
valid_data = lgb.Dataset(df_val[predictors], label=df_val[target], weight = df_val[weight], reference=train_data, free_raw_data=False)

In [131]:
#Create folds for cross-validation

folds = []
for fold in sorted(df_train[cv_col].unique()):
    val_ndx = df_train.index[df_train[cv_col] == fold]
    train_ndx = df_train.index[df_train[cv_col] != fold]
    folds.append((train_ndx, val_ndx))

In [132]:
%%time

#Use optuna to tune hyperparameters

import optuna

def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "boosting_type": "gbdt",
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log = True),
        "max_depth": trial.suggest_int("max_depth", 2, 6),
        "num_leaves": trial.suggest_int("num_leaves", 3, 50),
        "n_estimators": trial.suggest_int("n_estimators", 200, 800)
    }
    
    cv_result = lgb.cv(
        params,
        train_data,
        folds = folds,
        num_boost_round = 500,
        early_stopping_rounds = 10,
        seed = 42
    )
    #extract the best iterationn
    best_iter = len(cv_result["auc-mean"])
    
    #save it in the user atttribute to call later
    trial.set_user_attr("best_iteration", best_iter)
    
    return max(cv_result["auc-mean"])

study = optuna.create_study(direction = "maximize")
study.optimize(objective, n_trials = 30)

print("Best AUC:", study.best_value)
print("Best Parameters:", study.best_params)

[I 2025-06-28 21:06:53,020] A new study created in memory with name: no-name-e1fc80f5-8659-4b76-9bc2-1c5846aa20d7
C:\Users\scott\anaconda3\lib\site-packages\lightgbm\engine.py:573: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
C:\Users\scott\anaconda3\lib\site-packages\lightgbm\engine.py:577: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\scott\anaconda3\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


[LightGBM] [Info] Number of positive: 3665, number of negative: 11559
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001929 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 380
[LightGBM] [Info] Number of data points in the train set: 15224, number of used features: 11
[LightGBM] [Info] Number of positive: 3651, number of negative: 11533
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002146 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 380
[LightGBM] [Info] Number of data points in the train set: 15184, number of used features: 11
[LightGBM] [Info] Number of positive: 3656, number of negative: 11488
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of tes

[I 2025-06-28 21:06:58,274] Trial 0 finished with value: 0.9240663893956854 and parameters: {'learning_rate': 0.026163332487745477, 'max_depth': 2, 'num_leaves': 49, 'n_estimators': 667}. Best is trial 0 with value: 0.9240663893956854.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 3665, number of negative: 11559
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001936 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 380
[LightGBM] [Info] Number of data points in the train set: 15224, number of used features: 11
[LightGBM] [Info] Number of positive: 3651, number of negative: 11533
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002123 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 380
[LightGBM] [Info] Number of data points in the train set: 15184, number of used features: 11
[LightGBM] [Info] Number of positive: 3656, number of negative: 11488
[Ligh

C:\Users\scott\anaconda3\lib\site-packages\lightgbm\engine.py:573: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
C:\Users\scott\anaconda3\lib\site-packages\lightgbm\engine.py:577: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\scott\anaconda3\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


...

[LightGBM] [Info] Number of positive: 3665, number of negative: 11559
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002447 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 380
[LightGBM] [Info] Number of data points in the train set: 15224, number of used features: 11
[LightGBM] [Info] Number of positive: 3651, number of negative: 11533
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002262 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 380
[LightGBM] [Info] Number of data points in the train set: 15184, number of used features: 11
[LightGBM] [Info] Number of positive: 3656, number of negative: 11488
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of tes

[I 2025-06-28 21:08:52,577] Trial 28 finished with value: 0.9300721976183155 and parameters: {'learning_rate': 0.20660362931908305, 'max_depth': 4, 'num_leaves': 17, 'n_estimators': 531}. Best is trial 9 with value: 0.9311574515111604.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number 

[I 2025-06-28 21:08:55,288] Trial 29 finished with value: 0.9300647066536394 and parameters: {'learning_rate': 0.131325333336366, 'max_depth': 2, 'num_leaves': 14, 'n_estimators': 663}. Best is trial 9 with value: 0.9311574515111604.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Best AUC: 0.9311574515111604
Best Parameters: {'learning_rate': 0.07066150106298295, 'max_depth': 3, 'num_leaves': 37, 'n_estimators': 720}
CPU times: total: 9min 25s
Wall time: 2min 2s


In [133]:
#Train a model using the best hyperparameters

best_params = study.best_params.copy()

best_params.update({
    "objective": "binary",
    "metric": "auc",
    "verbosity": -1,
    "boosting_type": "gbdt"
})

best_iteration = study.best_trial.user_attrs["best_iteration"]

model = lgb.train(
    best_params,
    train_data,
    valid_sets = [train_data, valid_data],
    valid_names = ['train', 'valid'],
    num_boost_round = 1000,
    early_stopping_rounds = 50,
    verbose_eval = 50
)

Training until validation scores don't improve for 50 rounds


C:\Users\scott\anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
C:\Users\scott\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\scott\anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cntry', 'mrtl_sts', 'occptn', 'race', 'rlnshp', 'sex', 'workclass']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\scott\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and wil

[50]	train's auc: 0.921069	valid's auc: 0.915508
[100]	train's auc: 0.928414	valid's auc: 0.921147
[150]	train's auc: 0.93299	valid's auc: 0.92438
[200]	train's auc: 0.93651	valid's auc: 0.926765
[250]	train's auc: 0.939124	valid's auc: 0.92844
[300]	train's auc: 0.941142	valid's auc: 0.929512
[350]	train's auc: 0.942632	valid's auc: 0.930157
[400]	train's auc: 0.944014	valid's auc: 0.930551
[450]	train's auc: 0.945193	valid's auc: 0.93067
Early stopping, best iteration is:
[430]	train's auc: 0.944679	valid's auc: 0.930746


In [135]:
best_iter = model.best_iteration
print(f"Best Iteration: {best_iter}")

Best Iteration: 430


#### Test Data

In [145]:
#Prepare tthe test data for testinng the model

In [136]:
test['sex'] = test['sex'].astype('category')
test['workclass'] = test['workclass'].astype('category')
test['mrtl_sts'] = test['mrtl_sts'].astype('category')
test['occptn'] = test['occptn'].astype('category')
test['rlnshp'] = test['rlnshp'].astype('category')
test['race'] = test['race'].astype('category')
test['cntry'] = test['cntry'].astype('category')

In [137]:
test_data = lgb.Dataset(test[predictors], label=test[target], weight = test[weight], reference=train_data, free_raw_data=False)

In [139]:
p1 = model.predict(test[predictors])

In [144]:
from sklearn.metrics  import roc_auc_score

auc = roc_auc_score(test[target], p1, sample_weight = test[weight])
print(f"AUC: {auc:.2f}")
print(f"Gini: {2*auc - 1:.2f}")

AUC: 0.93
Gini: 0.86
